In [2]:
from Bio.Align import substitution_matrices
from Bio import Entrez, SeqIO
from Bio import Seq


1. Naloga

In [4]:
blosumMatrix = substitution_matrices.load("BLOSUM62")
def blosumFunction(x, y):
    if x == "-":
        return blosumMatrix["*"][y]
    elif y == "-":
        return blosumMatrix[x]["*"]
    else:
        return blosumMatrix[x][y]
def insertIntoString(string,char,position):
    return string[:position] + char + string[position:len(string)]
def global_allignment(seq1, seq2, scoring_function = blosumFunction):
    lengthSeq1 = len(seq1)
    lengthSeq2 = len(seq2)

    optMatrix = [[0 for j in range(lengthSeq2 + 1)] for i in range(lengthSeq1 + 1)]
    tracebackMatrix = [[('ins') if i == 0 else ('del') for j in range(lengthSeq2 + 1)] for i in range(lengthSeq1 + 1)]
    tracebackMatrix[0][0] = ('')
    for i in range(1, lengthSeq1 + 1):
        optMatrix[i][0] = i * scoring_function(seq1[i - 1],"-") 
    for j in range(1, lengthSeq2 + 1):
        optMatrix[0][j] = j * scoring_function("-",seq2[j - 1]) 

    for i in range(1, lengthSeq1 + 1):
        for j in range(1, lengthSeq2 + 1):
            allign = optMatrix[i-1][j-1] + scoring_function(seq1[i -1],seq2[j-1])
            delete = optMatrix[i - 1][j] + scoring_function("-", seq2[j - 1])
            insert = optMatrix[i][j-1] + scoring_function(seq1[i - 1],"-")
            optMatrix[i][j] = max(allign, delete, insert)
            
            if optMatrix[i][j] == allign:
                tracebackMatrix[i][j] = ('align')  
            elif optMatrix[i][j] == insert:
                tracebackMatrix[i][j] = ('ins') 
            elif optMatrix[i][j] == delete:
                tracebackMatrix[i][j] = ('del')    


    tracebackSteps = []
    nextOperation = tracebackMatrix[lengthSeq1][lengthSeq2]
    xCounter = lengthSeq1
    yCounter = lengthSeq2
    while True:
        tracebackSteps.append(nextOperation)
        if nextOperation== 'align':
            xCounter -= 1
            yCounter -= 1
        elif nextOperation == 'ins':
            yCounter -= 1
        elif nextOperation == 'del':
            xCounter -= 1
        nextOperation = tracebackMatrix[xCounter][yCounter]
        if nextOperation == (''):
            break
    tracebackSteps = tracebackSteps[::-1]

    for i in range(0,len(tracebackSteps)):
        action = tracebackSteps[i]
        if action == "allign":
            pass
        elif action == "ins":
            seq1 = insertIntoString(seq1,"-",i)
        elif action == "del":
            seq2 = insertIntoString(seq2,"-",i)

    return (optMatrix[lengthSeq1][lengthSeq2], seq1, seq2)  

2. Naloga

In [5]:

Entrez.email = "ls90321@student.uni-lj.si"
accession_codes = {
    # 6 known human coronaviruses
    "Human-SARS": "NC_004718",
    "Human-MERS": "NC_019843",
    "Human-HCoV-OC43": "NC_006213",
    "Human-HCoV-229E": "NC_002645",
    "Human-HCoV-NL63": "NC_005831",
    "Human-HCoV-HKU1": "NC_006577",
    
    # Bat
    "Bat-CoV MOP1": "EU420138",
    "Bat-CoV HKU8": "NC_010438",
    "Bat-CoV HKU2": "NC_009988",
    "Bat-CoV HKU5": "NC_009020",
    "Bat-CoV RaTG13": "MN996532",
    "Bat-CoV-ENT": "NC_003045",
    
    # Other animals
    "Hedgehog-CoV 2012-174/GER/2012": "NC_039207",
    "Pangolin-CoV MP789": "MT121216",
    "Rabbit-CoV HKU14": "NC_017083",
    "Duck-CoV isolate DK/GD/27/2014": "NC_048214",
    "Feline infectious peritonitis virus": "NC_002306",  # cat
    "Giraffe-CoV US/OH3/2003": "EF424623",
    "Murine-CoV MHV/BHKR_lab/USA/icA59_L94P/2012": "KF268338",  # mouse
    "Equine-CoV Obihiro12-2": "LC061274",  # horse
}
genes = {}
records = []
for key in accession_codes.keys():
    with Entrez.efetch(db="nucleotide", id = accession_codes[key], rettype="gbwithparts", retmode="FASTA") as handle:
        records.append(SeqIO.read(handle,"genbank"))
    for record in records:
        for gene in record.features:
            if gene.type == 'CDS':
                if 'gene' in gene.qualifiers.keys():
                    if gene.qualifiers['gene'] == ['S'] or gene.qualifiers['product'] == ['spike protein']:
                        genes[key] = gene
                elif gene.qualifiers['product'] == ['spike protein']:
                        genes[key] = gene

    records = []

covidSeq =""
firstLine = True
with open("data/sars_cov_2.fa") as f:
    for line in f.readlines():
        if not firstLine:
            whitoutBackslash = line.strip("\n")
            covidSeq += whitoutBackslash
        else:
            firstLine = False
sarsCovGenomeSpike = str(Seq.Seq(covidSeq[21562:25384]).translate())
allAnlignedScores = {}
for key in genes.keys():
    allAnlignedScores[key] = global_allignment(sarsCovGenomeSpike[0:len(sarsCovGenomeSpike)-1],genes[key].qualifiers['translation'][0])[0]


allAnlignedScores

three_closest_references = [
    "Bat-CoV RaTG13",
    "Pangolin-CoV MP789",
    "Human-SARS",
]

3. Naloga

In [186]:
def local_alignment(seq1, seq2, scoring_function = blosumFunction ):
    lengthSeq1 = len(seq1)
    lengthSeq2 = len(seq2)

    optMatrix = [[0 for j in range(lengthSeq2 + 1)] for i in range(lengthSeq1 + 1)]
    
    tracebackMatrix = [[('ins') if i == 0 else ('del') for j in range(lengthSeq2 + 1)] for i in range(lengthSeq1 + 1)]
    tracebackMatrix[0][0] = ('')
    for i in range(1, lengthSeq1 + 1):
        optMatrix[i][0] = max(i * scoring_function(seq1[i - 1],"-"),0) 
    for j in range(1, lengthSeq2 + 1):
        optMatrix[0][j] = max(j * scoring_function("-",seq2[j - 1]),0) 

    maxScore = -1
    maxScoreIndexS1 = 0
    maxScoreIndexS2 = 0
    for i in range(1, lengthSeq1 + 1):
        for j in range(1, lengthSeq2 + 1):
            allign = optMatrix[i-1][j-1] + scoring_function(seq1[i -1],seq2[j-1])
            delete = optMatrix[i - 1][j] + scoring_function("-", seq2[j - 1])
            insert = optMatrix[i][j-1] + scoring_function(seq1[i - 1],"-")
            elt = optMatrix[i][j] = max(allign, delete, insert, 0)
            
            if elt == allign or 0:
                tracebackMatrix[i][j] = ('align')  
            elif elt == insert:
                tracebackMatrix[i][j] = ('ins') 
            elif elt == delete:
                tracebackMatrix[i][j] = ('del')                 
            if elt > maxScore:
                maxScore = elt
                maxScoreIndexS1 = i
                maxScoreIndexS2 = j

    tracebackSteps = []
    nextOperation = tracebackMatrix[lengthSeq1][lengthSeq2]
    xCounter = maxScoreIndexS1
    yCounter = maxScoreIndexS2
    while True:
        tracebackSteps.append(nextOperation)
        if nextOperation == 'align':
            xCounter -= 1
            yCounter -= 1
        elif nextOperation == 'ins':
            yCounter -= 1
        elif nextOperation == 'del':
            xCounter -= 1
        nextOperation = tracebackMatrix[xCounter][yCounter]
        if nextOperation == ('') or optMatrix[xCounter][yCounter] == 0:
            break
    tracebackSteps = tracebackSteps[::-1]

    seq1 = seq1[xCounter:maxScoreIndexS1]
    seq2 = seq2[yCounter:maxScoreIndexS2]
    for i in range(0,len(tracebackSteps)):
        action = tracebackSteps[i]
        if action == "allign":
            pass
        elif action == "ins":
            seq1 = insertIntoString(seq1,"-",i)
        elif action == "del":
            seq2 = insertIntoString(seq2,"-",i)


    return (optMatrix[maxScoreIndexS1][maxScoreIndexS2], seq1, seq2)
  

4. Naloga

In [ ]:
orfsAndTranslations = {}
seq = str(Seq.Seq(covidSeq[11995:13483]).translate())
seq = seq[0:len(seq) - 1]
seq2 = str(Seq.Seq(covidSeq[26792:27191]).translate())
seq2 = seq2[0:len(seq2) - 1]
seq3 = str(Seq.Seq(covidSeq[23650:25384]).translate())
seq3 = seq3[0:len(seq3) - 1]
seq4 = str(Seq.Seq(covidSeq).reverse_complement()[len(covidSeq) - 667 : len(covidSeq) - 421].translate())
seq4 = seq4[0:len(seq4) - 1]
seq5 = str(Seq.Seq(covidSeq[9133:13483]).translate())
seq5 = seq2[0:len(seq5) - 1]
orfsAndTranslations["ORF-1"] = seq
orfsAndTranslations["ORF-2"] = seq2
orfsAndTranslations["ORF-3"] = seq3
orfsAndTranslations["ORF-4"] = seq4
orfsAndTranslations["ORF-5"] = seq5  

In [85]:
three_closest_references = [
    "Bat-CoV RaTG13",
    "Pangolin-CoV MP789",
    "Human-SARS",
]


In [147]:
relativeData = {"Bat-CoV RaTG13": [],"Pangolin-CoV MP789":[],"Human-SARS":[]}
records = []
for key in three_closest_references:
    with Entrez.efetch(db="nucleotide", id = accession_codes[key], rettype="gbwithparts", retmode="FASTA") as handle:
        records.append(SeqIO.read(handle,"genbank"))
    for record in records:
        for gene in record.features:
            if gene.type == 'CDS':
                relativeData[key].append(gene)
    records = []
    
relativeFunctions = {"Bat-CoV RaTG13": [],"Pangolin-CoV MP789":[],"Human-SARS":[]}
for key in relativeData.keys():
    for gene in relativeData[key]:
        data = gene
        relativeFunctions[key].append([data.qualifiers["translation"][0], data.qualifiers['product'][0]])
closestRelatives = {}
for key in orfsAndTranslations.keys():
    maxAl = -1
    for virusName in relativeFunctions.keys():
        for gene in relativeFunctions[virusName]:
            translatedVirus = gene[0]
            virusFunction = gene[1]
            aligment = local_alignment(translatedVirus, orfsAndTranslations[key])
            score = aligment[0]
            if score > maxAl:
                maxAl = score
                relativeName = virusName
                relativeFunction = virusFunction
                trLen = len(translatedVirus)
    closestRelatives[key] = [relativeName,relativeFunction, maxAl, trLen]      

In [ ]:
closestRelatives

6. Naloga

In [ ]:
three_closest_references = [ 
    "Human-MERS",
    "Bat-CoV HKU5",
    "Hedgehog-CoV 2012-174/GER/2012",
]
relativeData = {"Human-MERS": [],"Bat-CoV HKU5":[],"Hedgehog-CoV 2012-174/GER/2012":[]}

records = []
for key in three_closest_references:
    with Entrez.efetch(db="nucleotide", id = accession_codes[key], rettype="gbwithparts", retmode="FASTA") as handle:
        records.append(SeqIO.read(handle,"genbank"))

    for record in records:
        for gene in record.features:
            if gene.type == 'CDS':
                relativeData[key].append(gene)
    records = []
relativeFunctions = {"Human-MERS": [],"Bat-CoV HKU5":[],"Hedgehog-CoV 2012-174/GER/2012":[]}
for key in relativeData.keys():
    for gene in relativeData[key]:
        data = gene
        relativeFunctions[key].append([data.qualifiers["translation"][0], data.qualifiers['product'][0]])
closestRelatives = {}
for key in orfsAndTranslations.keys():
    maxAl = -1
    for virusName in relativeFunctions.keys():
        for gene in relativeFunctions[virusName]:
            translatedVirus = gene[0]
            virusFunction = gene[1]
            aligment = local_alignment(translatedVirus, orfsAndTranslations[key])
            score = aligment[0]
            if score > maxAl:
                maxAl = score
                relativeName = virusName
                relativeFunction = virusFunction
                trLen = len(translatedVirus)
    closestRelatives[key] = [relativeName,relativeFunction, maxAl, trLen]

In [183]:
closestRelatives

{'ORF-1': ['Hedgehog-CoV 2012-174/GER/2012',
  'ORF1ab polyprotein',
  1419.0,
  7150],
 'ORF-2': ['Human-MERS', 'membrane protein', 322.0, 219],
 'ORF-3': ['Human-MERS', 'spike protein', 1292.0, 1353],
 'ORF-4': ['Human-MERS', '1AB polyprotein', 59.0, 7078],
 'ORF-5': ['Human-MERS', 'membrane protein', 322.0, 219]}